In [1]:
# conda env test1

In [2]:
import torch
import torch.nn as nn
import pandas
import re, string
import emoji
from collections import Counter
import torchtext

from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler, random_split)
import tqdm
import numpy as np

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
data = pandas.read_csv('cyberbullying_tweets.csv')

In [4]:
data

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [5]:
data.loc[data['cyberbullying_type'] == 'other_cyberbullying']

,tweet_text,cyberbullying_type
23916,"@ikralla fyi, it looks like I was caught by it...",other_cyberbullying
23917,I need to just switch to an organization-based...,other_cyberbullying
23918,RMAed my monoprice. Shoddy power bricks on tho...,other_cyberbullying
23919,@murphy_slaw https://t.co/M8w8xnUnDL,other_cyberbullying
23920,@1Life0Continues i've got the code to interpre...,other_cyberbullying
...,...,...
31734,"@kufr666 @blockbot no, that's @oolon",other_cyberbullying
31735,@AriMelber why are you giving these idiots air...,other_cyberbullying
31736,I am right now watching Enforcers defend Chums...,other_cyberbullying
31737,✨✨✨ misandry is not a word iOS can autocomplet...,other_cyberbullying


In [6]:
labels = list(data.cyberbullying_type.unique())
labels2id = {label: i for i, label in enumerate(labels)}
print(labels2id)

{'not_cyberbullying': 0, 'gender': 1, 'religion': 2, 'other_cyberbullying': 3, 'age': 4, 'ethnicity': 5}


In [7]:
data['label'] = data.cyberbullying_type.map(labels2id)

In [8]:
#Clean emojis from text
def strip_emoji(text):
    return emoji.replace_emoji(text)
    # return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji
    
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    text = [word for word in text.split()]
    text = ' '.join(text)
    return text

#remove contractions
def decontract(text):
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def clean_hashtags(tweet):
    # new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as "&" and "$" present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

#Remove multiple sequential spaces
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)


#Then we apply all the defined functions in the following order
def deep_clean(text):
    text = strip_emoji(text)
    text = decontract(text)
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    return text

In [9]:
PADDING_VALUE = 256
data['cleaned_text'] =  [deep_clean(x).split()[:256] for x in data.tweet_text]

In [10]:
word_counts = Counter([item for sublist in data['cleaned_text'] for item in sublist])
vocab = torchtext.vocab.vocab(word_counts,
                              min_freq = 5,
                              specials = ['<pad>', '<unk>', ])
vocab.set_default_index(1)

VOCAB_SIZE = len(vocab)

In [11]:
tokenized_data = [torch.tensor(vocab(x), dtype = torch.int64) for x in data.cleaned_text]

labels = torch.tensor(data.label, dtype = torch.int64)

tokenized_data = nn.utils.rnn.pad_sequence(tokenized_data, batch_first=True)

# bag of words input. frequency of token
bow_tokenized_data = torch.zeros(tokenized_data.shape[0], VOCAB_SIZE)
for i in range(tokenized_data.shape[0]):
  tweet = tokenized_data[i,tokenized_data[i].nonzero().flatten()]
  bow_tokenized_data[i].put_(tweet, torch.ones_like(tweet).float(), accumulate=True)

bow_tokenized_data = nn.utils.rnn.pad_sequence(bow_tokenized_data, batch_first=True)

In [12]:
TRAIN_SPLIT = int(0.8 * len(labels))
VAL_TEST_SPLIT = len(labels) - TRAIN_SPLIT
VAL_SPLIT = VAL_TEST_SPLIT//2
TEST_SPLIT = VAL_TEST_SPLIT - VAL_SPLIT

bow_cnn_dataset = torch.utils.data.TensorDataset(tokenized_data, labels)
bow_dataset = torch.utils.data.TensorDataset(bow_tokenized_data, labels)

In [13]:
train, val_test = torch.utils.data.random_split(bow_cnn_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT])
val, test = torch.utils.data.random_split(val_test, [VAL_SPLIT, TEST_SPLIT])

print(len(train), len(val), len(test))

bow_train, bow_val_test = torch.utils.data.random_split(bow_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT])
bow_val, bow_test = torch.utils.data.random_split(bow_val_test, [VAL_SPLIT, TEST_SPLIT])

print(len(bow_train), len(bow_val), len(bow_test))

38153 4769 4770
38153 4769 4770


In [14]:
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(train, BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val, BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test, BATCH_SIZE)

bow_train_loader = torch.utils.data.DataLoader(bow_train, BATCH_SIZE)
bow_val_loader = torch.utils.data.DataLoader(bow_val, BATCH_SIZE)
bow_test_loader = torch.utils.data.DataLoader(bow_test, BATCH_SIZE)

# BOW

In [15]:
class MLP(nn.Module):
  def __init__(self, hidden_dim, dropout, device, outputs):
    super(MLP, self).__init__()

    self.hidden_dim = hidden_dim
    self.dropout = dropout
    self.device = device
    self.outputs = outputs
    
    self.l1 = nn.LazyLinear(hidden_dim).to(self.device)
    self.a1 = nn.ReLU().to(self.device)
    self.d1 = nn.Dropout().to(self.device)
    self.l2 = nn.LazyLinear(self.outputs).to(self.device)

    self.loss = nn.CrossEntropyLoss()
  
  def forward(self, x, y):
    x = self.l1(x).to(self.device)
    x = self.a1(x).to(self.device)
    x = self.d1(x).to(self.device)
    x = self.l2(x).to(self.device)

    loss = self.loss(x, y).to(device)

    return x, loss

  def get_predictions(self, loader):
    with torch.no_grad():
      preds_all = []
      labels_all = []
      accuracies = []

      for tweets, labels in loader:
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        self.eval()

        logits, loss = self.forward(tweets, labels)

        _, predictions = torch.max(logits, axis=1)

        preds_all += list(predictions.cpu())
        labels_all += list(labels.cpu())


        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

    return preds_all, labels_all
  
  def train_model(self, train_loader, val_loader, test_loader, optimizer):
    with tqdm.notebook.tqdm(
      train_loader,
      unit="batch",
      total=len(train_loader)) as batch_iterator:

      total_loss = 0.0
      accuracies = []

      for iteration, data in enumerate(batch_iterator, start=1):
        tweets, labels = data
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        optimizer.zero_grad()
        self.zero_grad()
        self.train()

        logits, loss = self.forward(tweets, labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        _, predictions = torch.max(logits, axis=1)
        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

        batch_iterator.set_postfix(mean_loss=total_loss / iteration, current_loss=loss.item(), accuracy = accuracy, mean_accuracy = np.mean(accuracies))

        if iteration % 500 == 0:
          pred, label = self.get_predictions(val_loader)
          print("########## VAL ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          pred, label = self.get_predictions(test_loader)
          print("########## TEST ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)
  


In [16]:
hidden_dim = 128
dropout = 0.
learning_rate = 1e-04
epochs = 15
outputs = len(labels2id.items())

mlp = MLP(hidden_dim, dropout, device, outputs)
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

for i in range(epochs):
  mlp.train_model(bow_train_loader, bow_val_loader, bow_test_loader, optimizer)

C:\Users\bojun\anaconda3\envs\test1\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.7372614803942126
0.7372614803942126
              precision    recall  f1-score   support

           0     0.2548    0.7256    0.3771       277
           1     0.8075    0.7497    0.7775       867
           2     0.9782    0.6955    0.8130      1097
           3     0.4358    0.5536    0.4877       625
           4     0.9859    0.7744    0.8675       993
           5     0.9586    0.8648    0.9093       910

    accuracy                         0.7373      4769
   macro avg     0.7368    0.7273    0.7053      4769
weighted avg     0.8319    0.7373    0.7683      4769

########## TEST ###########
0.7268343815513627
0.7268343815513627
              precision    recall  f1-score   support

           0     0.2400    0.7283    0.3611       265
           1     0.8088    0.7335    0.7693       848
           2     0.9690    0.6890    0.8053      1180
           3     0.4021    0.5126    0.4507       597
           4     0.9612    0.7767    0.8592      1021
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8066680645837702
0.80666806458377
              precision    recall  f1-score   support

           0     0.4474    0.6281    0.5226       562
           1     0.7801    0.8870    0.8301       708
           2     0.9500    0.8982    0.9234       825
           3     0.6990    0.5640    0.6243       984
           4     0.9872    0.9200    0.9524       837
           5     0.9744    0.9379    0.9558       853

    accuracy                         0.8067      4769
   macro avg     0.8064    0.8059    0.8014      4769
weighted avg     0.8247    0.8067    0.8115      4769

########## TEST ###########
0.79979035639413
0.79979035639413
              precision    recall  f1-score   support

           0     0.4055    0.6293    0.4932       518
           1     0.7633    0.9017    0.8268       651
           2     0.9523    0.8800    0.9147       908
           3     0.7201    0.5315    0.6116      1031
           4     0.9685    0.9173    0.9422       871
       

...

########## VAL ###########
0.8339274481023276
0.8339274481023276
              precision    recall  f1-score   support

           0     0.5158    0.6167    0.5618       660
           1     0.8335    0.8947    0.8630       750
           2     0.9500    0.9549    0.9524       776
           3     0.7380    0.5949    0.6588       985
           4     0.9846    0.9808    0.9827       783
           5     0.9793    0.9865    0.9829       815

    accuracy                         0.8339      4769
   macro avg     0.8336    0.8381    0.8336      4769
weighted avg     0.8385    0.8339    0.8338      4769

########## TEST ###########
0.8377358490566038
0.8377358490566038
              precision    recall  f1-score   support

           0     0.5075    0.6528    0.5710       625
           1     0.8336    0.8890    0.8604       721
           2     0.9452    0.9474    0.9463       837
           3     0.7792    0.5996    0.6777       989
           4     0.9709    0.9662    0.9686       829
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.832459635143636
0.832459635143636
              precision    recall  f1-score   support

           0     0.5171    0.6108    0.5601       668
           1     0.8335    0.8911    0.8614       753
           2     0.9526    0.9550    0.9538       778
           3     0.7280    0.5922    0.6531       976
           4     0.9833    0.9808    0.9821       782
           5     0.9781    0.9889    0.9835       812

    accuracy                         0.8325      4769
   macro avg     0.8321    0.8365    0.8323      4769
weighted avg     0.8362    0.8325    0.8322      4769

########## TEST ###########
0.8358490566037736
0.8358490566037736
              precision    recall  f1-score   support

           0     0.5100    0.6426    0.5687       638
           1     0.8336    0.8854    0.8587       724
           2     0.9452    0.9474    0.9463       837
           3     0.7648    0.5982    0.6713       973
           4     0.9709    0.9662    0.9686       829
   

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8316208848815265
0.8316208848815265
              precision    recall  f1-score   support

           0     0.5298    0.5989    0.5622       698
           1     0.8323    0.8921    0.8612       751
           2     0.9526    0.9562    0.9544       777
           3     0.7103    0.5943    0.6472       949
           4     0.9833    0.9808    0.9821       782
           5     0.9793    0.9901    0.9847       812

    accuracy                         0.8316      4769
   macro avg     0.8313    0.8354    0.8320      4769
weighted avg     0.8331    0.8316    0.8309      4769

########## TEST ###########
0.8354297693920335
0.8354297693920335
              precision    recall  f1-score   support

           0     0.5199    0.6372    0.5726       656
           1     0.8309    0.8863    0.8577       721
           2     0.9452    0.9474    0.9463       837
           3     0.7543    0.5992    0.6678       958
           4     0.9709    0.9662    0.9686       829
 

# CNN

In [17]:
from torch.nn.modules.conv import Conv1d
class Conv1dBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=2, padding=0, max_pool=2, dropout=0.1):
    super(Conv1dBlock, self).__init__()

    self.max_pool = max_pool
    self.l1 = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding = padding)
    self.l2 = nn.ReLU()
    self.l3 = nn.MaxPool1d(kernel_size=self.max_pool, stride=1, padding=0)
    self.l4 = nn.Dropout(dropout)
  
  def __call__(self, x):
    x = self.l1(x)
    x = self.l2(x)
    if self.max_pool > 0:
      x = self.l3(x)
    x = self.l4(x)
    return x

class ConvNet(nn.Module):
  def __init__(self, hidden_dim, dropout, device, outputs, embedding_dim, kernel_size=2, max_pool=2):
    super(ConvNet, self).__init__()

    self.conv_dim = conv_dim
    self.hidden_dim = hidden_dim
    self.dropout = dropout
    self.device = device
    self.outputs = outputs
    self.embedding_dim = embedding_dim
    self.max_pool = max_pool
    self.kernel_size = kernel_size

    self.e = nn.Embedding(num_embeddings = VOCAB_SIZE, embedding_dim=self.embedding_dim).to(self.device)

    self.c1 = Conv1dBlock(in_channels=self.embedding_dim, out_channels=self.conv_dim, kernel_size=self.kernel_size, padding=0, max_pool=self.max_pool, dropout=0.1).to(self.device)
    self.c2 = Conv1dBlock(in_channels=self.conv_dim, out_channels=self.conv_dim, kernel_size=self.kernel_size, padding=0, max_pool=self.max_pool, dropout=0.1).to(self.device)
    self.c3 = Conv1dBlock(in_channels=self.conv_dim, out_channels=self.conv_dim, kernel_size=self.kernel_size, padding=0, max_pool=self.max_pool, dropout=0.1).to(self.device)

    self.l1 = nn.LazyLinear(hidden_dim).to(self.device)
    self.a1 = nn.ReLU().to(self.device)
    self.d1 = nn.Dropout().to(self.device)
    self.l2 = nn.LazyLinear(self.outputs).to(self.device)

    self.loss = nn.CrossEntropyLoss()
  
  def forward(self, x, y):
    embeddings = self.e(x)
    C, L, I = embeddings.shape
    x = embeddings.reshape(C, I, L)
    x = self.c1(x).to(self.device)
    x = self.c2(x).to(self.device)
    x = self.c3(x).to(self.device)
    x = x.flatten(1)
    x = self.l1(x).to(self.device)
    x = self.a1(x).to(self.device)
    x = self.d1(x).to(self.device)
    x = self.l2(x).to(self.device)
    # x = x[:,0,:]
    loss = self.loss(x, y).to(device)

    return x, loss

  def get_predictions(self, loader):
    with torch.no_grad():
      preds_all = []
      labels_all = []
      accuracies = []

      for tweets, labels in loader:
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        self.eval()

        logits, loss = self.forward(tweets, labels)

        _, predictions = torch.max(logits, axis=1)

        preds_all += list(predictions.cpu())
        labels_all += list(labels.cpu())


        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

    return preds_all, labels_all
  
  def train_model(self, train_loader, val_loader, test_loader, optimizer):
    with tqdm.notebook.tqdm(
      train_loader,
      unit="batch",
      total=len(train_loader)) as batch_iterator:

      total_loss = 0.0
      accuracies = []

      for iteration, data in enumerate(batch_iterator, start=1):
        tweets, labels = data
        tweets = tweets.to(self.device)
        labels = labels.to(self.device)

        optimizer.zero_grad()
        self.zero_grad()
        self.train()


        logits, loss = self.forward(tweets, labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        _, predictions = torch.max(logits, axis=1)
        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

        batch_iterator.set_postfix(mean_loss=total_loss / iteration, current_loss=loss.item(), accuracy = accuracy, mean_accuracy = np.mean(accuracies))

        if iteration % 500 == 0:
          pred, label = self.get_predictions(val_loader)
          print("########## VAL ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          pred, label = self.get_predictions(test_loader)
          print("########## TEST ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)
  


In [18]:

conv_dim = 32
hidden_dim = 128
dropout = 0.1
learning_rate = 5e-04
epochs = 15
outputs = len(labels2id.items())
embedding_dim = 128

convnet = ConvNet(hidden_dim, dropout, device, outputs, embedding_dim, kernel_size=8, max_pool=2)
optimizer = torch.optim.Adam(convnet.parameters(), lr=learning_rate)

for i in range(epochs):
  convnet.train_model(train_loader, val_loader, test_loader, optimizer)

C:\Users\bojun\anaconda3\envs\test1\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.6756133361291675
0.6756133361291675
              precision    recall  f1-score   support

           0     0.4558    0.4623    0.4591       770
           1     0.7052    0.6932    0.6991       828
           2     0.7545    0.7393    0.7468       794
           3     0.4494    0.4594    0.4544       764
           4     0.8269    0.9253    0.8734       723
           5     0.8499    0.7697    0.8078       890

    accuracy                         0.6756      4769
   macro avg     0.6736    0.6749    0.6734      4769
weighted avg     0.6776    0.6756    0.6758      4769

########## TEST ###########
0.6631027253668763
0.6631027253668763
              precision    recall  f1-score   support

           0     0.4130    0.4661    0.4379       723
           1     0.6676    0.6402    0.6536       781
           2     0.7805    0.6987    0.7373       896
           3     0.4724    0.4643    0.4683       812
           4     0.8170    0.9262    0.8681       718
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.728244915076536
0.7282449150765359
              precision    recall  f1-score   support

           0     0.7324    0.4370    0.5474      1309
           1     0.6978    0.8806    0.7786       645
           2     0.7931    0.9168    0.8504       673
           3     0.2650    0.4882    0.3436       424
           4     0.9184    0.9346    0.9264       795
           5     0.9504    0.8299    0.8861       923

    accuracy                         0.7282      4769
   macro avg     0.7262    0.7478    0.7221      4769
weighted avg     0.7679    0.7282    0.7320      4769

########## TEST ###########
0.7182389937106918
0.7182389937106918
              precision    recall  f1-score   support

           0     0.7243    0.4434    0.5500      1333
           1     0.6542    0.8305    0.7319       590
           2     0.8180    0.8877    0.8514       739
           3     0.2481    0.4681    0.3243       423
           4     0.9201    0.9328    0.9264       803
  

...

########## VAL ###########
0.8098133780666806
0.8098133780666806
              precision    recall  f1-score   support

           0     0.5595    0.5483    0.5539       797
           1     0.7789    0.9175    0.8425       691
           2     0.9422    0.9349    0.9385       784
           3     0.6389    0.5664    0.6005       881
           4     0.9629    0.9774    0.9701       797
           5     0.9677    0.9524    0.9600       819

    accuracy                         0.8098      4769
   macro avg     0.8084    0.8162    0.8109      4769
weighted avg     0.8064    0.8098    0.8069      4769

########## TEST ###########
0.7987421383647799
0.7987421383647799
              precision    recall  f1-score   support

           0     0.5233    0.5553    0.5388       769
           1     0.7530    0.9038    0.8216       624
           2     0.9501    0.9338    0.9419       816
           3     0.6366    0.5275    0.5769       963
           4     0.9595    0.9726    0.9660       803
 

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8096036905011533
0.8096036905011533
              precision    recall  f1-score   support

           0     0.4904    0.5682    0.5265       674
           1     0.7789    0.9337    0.8493       679
           2     0.9319    0.9319    0.9319       778
           3     0.7157    0.5406    0.6160      1034
           4     0.9666    0.9775    0.9720       800
           5     0.9653    0.9677    0.9665       804

    accuracy                         0.8096      4769
   macro avg     0.8081    0.8199    0.8104      4769
weighted avg     0.8123    0.8096    0.8069      4769

########## TEST ###########
0.8
0.8000000000000002
              precision    recall  f1-score   support

           0     0.4718    0.5755    0.5185       669
           1     0.7503    0.9065    0.8210       620
           2     0.9377    0.9435    0.9406       797
           3     0.7155    0.5196    0.6020      1099
           4     0.9558    0.9786    0.9671       795
           5    

  0%|          | 0/597 [00:00<?, ?batch/s]

########## VAL ###########
0.8056196267561334
0.8056196267561334
              precision    recall  f1-score   support

           0     0.4942    0.5522    0.5216       699
           1     0.7838    0.9154    0.8445       697
           2     0.9344    0.9491    0.9417       766
           3     0.6620    0.5391    0.5943       959
           4     0.9740    0.9563    0.9651       824
           5     0.9752    0.9539    0.9644       824

    accuracy                         0.8056      4769
   macro avg     0.8039    0.8110    0.8053      4769
weighted avg     0.8070    0.8056    0.8040      4769

########## TEST ###########
0.8064989517819706
0.8064989517819706
              precision    recall  f1-score   support

           0     0.5233    0.5882    0.5538       726
           1     0.7583    0.9045    0.8250       628
           2     0.9352    0.9518    0.9434       788
           3     0.6754    0.5401    0.6002       998
           4     0.9730    0.9588    0.9659       826
 

# Transformer Models

In [19]:
data

,tweet_text,cyberbullying_type,label,cleaned_text
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0,"[in, other, words, katandandre, your, food, wa..."
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0,"[why, is, aussietv, so, white, mkr, theblock, ..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0,"[a, classy, whore, or, more, red, velvet, cupc..."
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0,"[meh, p, thanks, for, the, heads, up, but, not..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0,"[this, is, an, isis, account, pretending, to, ..."
...,...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity,5,"[black, ppl, are, not, expected, to, do, anyth..."
47688,Turner did not withhold his disappointment. Tu...,ethnicity,5,"[turner, did, not, withhold, his, disappointme..."
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,5,"[i, swear, to, god, this, dumb, nigger, bitch,..."
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,5,"[yea, fuck, you, rt, if, youre, a, nigger, fuc..."


In [20]:
BATCH_SIZE = 50
del convnet
del mlp

In [21]:
model_type = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_type, use_fast=False)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\bojun\anaconda3\envs\test1\lib\site-packages\huggingface_hub\file_download.py:125: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bojun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [22]:
tokenized_text = list(data['cleaned_text'])
tokenized_text = [' '.join(x) for x in tokenized_text]
tokenized_text = tokenizer(tokenized_text, padding = 'max_length', max_length = PADDING_VALUE, truncation=True, add_special_tokens=True)
x_ids = torch.tensor([f for f in tokenized_text.input_ids], dtype=torch.long)
x_mask = torch.tensor([f for f in tokenized_text.attention_mask], dtype=torch.long)
label_tensor = torch.tensor(data.label)

In [23]:
tensor_dataset = TensorDataset(x_ids, x_mask, label_tensor)

In [24]:
train, val_test = random_split(tensor_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT], generator=torch.Generator().manual_seed(13))
val, test = random_split(val_test, [VAL_SPLIT, TEST_SPLIT], generator=torch.Generator().manual_seed(13))

print(len(train), len(val), len(test))

38153 4769 4770


In [25]:
train_loader = DataLoader(train, batch_size=BATCH_SIZE)
val_loader = DataLoader(val, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

In [26]:
class transformer_classifier(nn.Module):
  def __init__(self, model_type, hidden_dim, dropout, device ):
      super(transformer_classifier, self).__init__()

      self.device = device
      self.hidden_dim = hidden_dim
      self.LABELS_NUM = len(labels2id.items())

      self.lm = AutoModel.from_pretrained(model_type).to(self.device)

      config = config = AutoConfig.from_pretrained(model_type, num_labels = self.LABELS_NUM)
      self.lm_sequence_classification = AutoModelForSequenceClassification.from_pretrained(model_type, config=config).to(self.device)

      self.l1 = nn.LazyLinear(hidden_dim).to(self.device)
      self.a1 = nn.GELU().to(self.device)
      self.d1 = nn.Dropout(dropout).to(self.device)
      self.l2 = nn.LazyLinear(self.LABELS_NUM).to(self.device)

      self.loss = nn.CrossEntropyLoss()


  def forward(self, ids, masks, labels, mode):
    if mode:
      output = self.lm_sequence_classification(ids, masks, labels=labels)
      return output.logits.to(self.device), output.loss.to(self.device)
    else:
      x = self.lm(ids, masks).last_hidden_state[:, 0].to(self.device)
      x = self.l1(x).to(self.device)
      x = self.a1(x).to(self.device)
      x = self.d1(x).to(self.device)
      x = self.l2(x).to(self.device)

      loss = self.loss(x, labels).to(self.device)

      return x, loss

  def get_predictions(self, loader, mode):
    with torch.no_grad():
      with tqdm.notebook.tqdm(
      loader,
      unit="batch",
      total=len(loader)) as batch_iterator:

        preds_all = []
        labels_all = []
        accuracies = []

        for iteration, data in enumerate(batch_iterator, start=1):
          ids, masks, labels = data
          ids = ids.to(self.device)
          masks = masks.to(self.device)
          labels = labels.to(self.device)

          self.eval()

          logits, loss = self.forward(ids, masks, labels, mode)

          _, predictions = torch.max(logits, axis=1)

          preds_all += list(predictions.cpu())
          labels_all += list(labels.cpu())


          accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
          accuracies.append(accuracy)

          batch_iterator.set_postfix(accuracy = accuracy, mean_accuracy = np.mean(accuracies))
    return preds_all, labels_all

  def train_model(self, train_loader, val_loader, test_loader, mode, optimizer):
    with tqdm.notebook.tqdm(
      train_loader,
      unit="batch",
      total=len(train_loader)) as batch_iterator:

      total_loss = 0.0
      accuracies = []

      for iteration, data in enumerate(batch_iterator, start=1):
        ids, masks, labels = data
        ids = ids.to(self.device)
        masks = masks.to(self.device)
        labels = labels.to(self.device)

        optimizer.zero_grad()
        self.zero_grad()
        self.train()

        logits, loss = self.forward(ids, masks, labels, mode)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        _, predictions = torch.max(logits, axis=1)
        accuracy = accuracy_score(list(labels.cpu()), list(predictions.cpu()))
        accuracies.append(accuracy)

        batch_iterator.set_postfix(mean_loss=total_loss / iteration, current_loss=loss.item(), accuracy = accuracy, mean_accuracy = np.mean(accuracies))

        if iteration % 400 == 0:
          pred, label = self.get_predictions(val_loader, mode)
          print("########## VAL ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          pred, label = self.get_predictions(test_loader, mode)
          print("########## TEST ###########")
          print(accuracy_score(pred, label))
          print(f1_score(pred, label, average='micro'))
          print(classification_report(pred, label, zero_division=0, digits = 4),)

          



In [27]:
hidden_dim = 128
dropout = 0.2
learning_rate = 1e-04
epochs = 5

classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, False, optimizer)

classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, True, optimizer)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerN

  0%|          | 0/764 [00:00<?, ?batch/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 150.00 MiB (GPU 0; 8.00 GiB total capacity; 7.05 GiB already allocated; 0 bytes free; 7.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
model_type

'bert-base-uncased'

In [29]:
model_type = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_type, use_fast=False)
tokenized_text = list(data['cleaned_text'])
tokenized_text = [' '.join(x) for x in tokenized_text]
tokenized_text = tokenizer(tokenized_text, padding = 'max_length', max_length = PADDING_VALUE, truncation=True, add_special_tokens=True)
x_ids = torch.tensor([f for f in tokenized_text.input_ids], dtype=torch.long)
x_mask = torch.tensor([f for f in tokenized_text.attention_mask], dtype=torch.long)
label_tensor = torch.tensor(data.label)
tensor_dataset = TensorDataset(x_ids, x_mask, label_tensor)
train, val_test = random_split(tensor_dataset, [TRAIN_SPLIT, VAL_TEST_SPLIT], generator=torch.Generator().manual_seed(13))
val, test = random_split(val_test, [VAL_SPLIT, TEST_SPLIT], generator=torch.Generator().manual_seed(13))
train_loader = DataLoader(train, batch_size=BATCH_SIZE)
val_loader = DataLoader(val, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [30]:
classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, False, optimizer)

classifier = transformer_classifier(model_type, hidden_dim, dropout, device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

for i in range(epochs):
  classifier.train_model(train_loader, val_loader, test_loader, True, optimizer)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 7.18 GiB already allocated; 0 bytes free; 7.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF